In [1]:
import pyarrow as pa
import pandas as pd
import numpy as np
import dotenv
import pyodbc
import os
from urllib.parse import quote_plus
from sqlalchemy import create_engine, Column, Integer, String, text
from sqlalchemy.ext.declarative import declarative_base
from geoalchemy2 import Geometry
from sqlalchemy.orm import sessionmaker

# CONNECTION

In [2]:
dotenv.load_dotenv('./.env', override=True)

DRIVER = os.getenv("DRIVER")
SERVER = os.getenv("SERVER")
if SERVER.startswith("tcp:"):  # Nettoyage de l'adresse serveur
    SERVER = SERVER.replace("tcp:", "")
print(SERVER)
DATABASE = os.getenv("DATABASE")
UID = os.getenv("UID")
PWD = os.getenv("PWD")
ENCRYPT = os.getenv("ENCRYPT")
TRUSTSERVERCERTIFICATE = os.getenv("TRUSTSERVERCERTIFICATE")
CONNECTION_TIMEOUT = os.getenv("CONNECTION_TIMEOUT")

# Establish the connection
# conn = pyodbc.connect(f'''
#                       Driver={DRIVER};
#                       Server={SERVER},1433;
#                       Database={DATABASE};
#                       Uid={UID};
#                       Pwd={PWD};
#                       Encrypt={ENCRYPT};
#                       TrustServerCertificate={TRUSTSERVERCERTIFICATE};
#                       Connection Timeout={CONNECTION_TIMEOUT};''')

# Encodage des paramètres pour SQLAlchemy
connection_string = (
    f"mssql+pyodbc://{quote_plus(UID)}:{quote_plus(PWD)}@{SERVER},{1433}/{DATABASE}"
    f"?driver={quote_plus(DRIVER)}&encrypt={ENCRYPT}&TrustServerCertificate={TRUSTSERVERCERTIFICATE}"
    f"&timeout={CONNECTION_TIMEOUT}"
)

# Création du moteur SQLAlchemy
engine = create_engine(connection_string)

adventureworks-server-hdf.database.windows.net


# READ ALL TABLES

In [3]:
import os

query = """
SELECT *
  FROM INFORMATION_SCHEMA.TABLES
 WHERE table_schema IN ('Person', 'Production', 'Sales')
"""

df_tables = pd.read_sql_query(query, engine)

# Create the output directory if it doesn't exist
output_dir = './output/SQL-server/'
os.makedirs(output_dir, exist_ok=True)

# Loop through each table and save its data to a CSV file
# for index, row in df.iterrows():
#   table_schema = row['TABLE_SCHEMA']
#   table_name = row['TABLE_NAME']
#   print(index, table_schema, table_name)
#   table_query = f"SELECT * FROM {table_schema}.{table_name}"
#   table_df = pd.read_sql_query(table_query, engine)
#   output_file = os.path.join(output_dir, f"{table_schema}.{table_name}.csv")
#   table_df.to_csv(output_file, index=False)


# Fonction pour obtenir les colonnes et leurs types pour une table donnée
def get_table_columns(schema, table, engine):
    columns_query = """
    SELECT COLUMN_NAME, DATA_TYPE
      FROM INFORMATION_SCHEMA.COLUMNS
     WHERE TABLE_SCHEMA = :schema AND TABLE_NAME = :table
    """
    return pd.read_sql_query(text(columns_query), engine, params={"schema": schema, "table": table})

# Boucle à travers chaque table et sauvegarde ses données dans un fichier CSV
for index, row in df_tables.iterrows():
    table_schema = row['TABLE_SCHEMA']
    table_name = row['TABLE_NAME']
    # print(f"Traitement de la table {index}: {table_schema}.{table_name}")
    
    # Obtenir les colonnes et types de la table
    df_columns = get_table_columns(table_schema, table_name, engine)
    
    # Construire la clause SELECT en gérant les colonnes spatiales
    select_columns = []
    for _, col in df_columns.iterrows():
        column_name = col['COLUMN_NAME']
        data_type = col['DATA_TYPE']
        if data_type in ('geometry', 'geography'):
            # Convertir les colonnes spatiales en WKT
            select_columns.append(f"{column_name}.STAsText() AS {column_name}_WKT")
        else:
            select_columns.append(column_name)
    
    select_clause = ", ".join(select_columns)
    table_query = f"SELECT {select_clause} FROM {table_schema}.{table_name}"
    
    # Lire les données de la table
    try:
        table_df = pd.read_sql_query(table_query, engine)
    except Exception as e:
        print(f"Erreur lors de la lecture de la table {table_schema}.{table_name}: {e}")
        continue
    
    # Définir le chemin du fichier de sortie
    output_file = os.path.join(output_dir, f"{table_schema}.{table_name}.csv")
    
    # Sauvegarder les données dans un fichier CSV
    table_df.to_csv(output_file, index=False)
    # print(f"Table {table_schema}.{table_name} sauvegardée dans {output_file}")

print("Exportation terminée.")

Erreur lors de la lecture de la table Production.ProductProductPhoto: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Incorrect syntax near the keyword 'Primary'. (156) (SQLExecDirectW)")
[SQL: SELECT ProductID, ProductPhotoID, Primary, ModifiedDate FROM Production.ProductProductPhoto]
(Background on this error at: https://sqlalche.me/e/20/f405)
Erreur lors de la lecture de la table Production.ProductDocument: (pyodbc.ProgrammingError) ('ODBC SQL type -151 is not yet supported.  column-index=1  type=-151', 'HY106')
(Background on this error at: https://sqlalche.me/e/20/f405)
Erreur lors de la lecture de la table Production.Document: (pyodbc.ProgrammingError) ('ODBC SQL type -151 is not yet supported.  column-index=0  type=-151', 'HY106')
(Background on this error at: https://sqlalche.me/e/20/f405)
Erreur lors de la lecture de la table Sales.SalesTerritory: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL 

In [3]:
# Fermer l'engine pour libérer les ressources
engine.dispose()

NameError: name 'engine' is not defined